# TP2 Robotica Movel - Potential Fields

## Alunos
- Bernardo Nogueira Borges 2020006396
- Daniele Cassia Silva Diniz 2020076874


### Importando Libs

In [16]:
# from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import networkx as nx
import numpy as np

# client = RemoteAPIClient()
# sim = client.require('sim')
np.set_printoptions(precision=3,suppress=True)

### Definindo Parametros

### Obtendo Obstaculos da Imagem

### Criando Gradiente

## Funções Potencial

### Força de Atração
$$F_{att}(q) = k_{att} \cdot (q_{goal} - q)$$

In [10]:
def ForceAttraction(q, q_goal, katt=.01):
    return katt * (q_goal - q)

### Força de repulsão
$$F_{rep,i}(q) = -\nabla U_{rep,i}(q)$$
$$=\begin{cases}
  \begin{aligned}
    \frac{k_{rep,i}}{\rho_i^2(q)}\left( \frac{1}{\rho_i(q)} - \frac{1}{\rho_{0,i}} \right)^{\gamma-1} \frac{q - q_{obs,i}}{\rho_i(q)} &, \textnormal{se } \rho_i(q) \le \rho_{0,i}\\
    0&, \textnormal{se } \rho_i(q) \gt \rho_{0,i}\\
  \end{aligned}
\end{cases}
$$

In [11]:
def ForceRepulsion(q, obs, R=3, krep=.1):
    # Obstaculo: (x,y,r)
    v = q - obs[0:2]
    d = np.linalg.norm(v, axis=1) - obs[2]
    d = d.reshape((len(v), 1))

    rep = (1/d**2)*((1/d)-(1/R))*(v/d)

    invalid = np.squeeze(d > R)
    rep[invalid, :] = 0

    return krep * rep


### Força Resultante 
$$F(q)  = F_{att}(q) + \sum_{i=1}^{p}{F_{rep,i}(q)}$$

In [18]:
# TODO: Deal with laser readings ?
def ForceResult(q,q_goal,laser_reads):
    ans = ForceAttraction(q,q_goal) 
    for i in range(len(laser_reads)):
        ans += ForceRepulsion(q,laser_reads[i])
    return ans

# ? Problem with the `q`... what type is it?
# print(ForceResult(np.array([0,0]),
#                   np.array([1,1]),
#                   np.array([[1,1],[1,2]])))

### Controladores
- [De Luca e Oriolo, 1994](https://www.researchgate.net/publication/225543929_Control_of_Wheeled_Mobile_Robots_An_Experimental_Overview)
$$v = k_{\rho}(\dot x \cos \theta + \dot y \sin \theta)$$
$$\omega = k_{\theta}(atan2(\dot y,\dot x) - \theta)$$

### Movendo o Robo